In [1]:
import json
import pandas as pd
import numpy as np
from scipy.stats import wilcoxon

In [2]:
corpus = [ json.loads(line) for line in open("controlled_generation_comparison_results_galactica6.7B-PPO_ChatGPT.jsonl") ] 
corpus = [ item for item in corpus if "user_preference" in item ]

In [3]:
len(corpus)

105

In [4]:
model_A = corpus[0]["generated_citations"][0]["model"]
model_B = corpus[0]["generated_citations"][1]["model"]

In [5]:
all_metrics = ["IAS","KR","FS","Similarity"]

In [6]:
results = {}
for metric in all_metrics:
    results[metric] = {
        model_A:[],
        model_B:[],
        "Neutral":[]
    }

In [7]:
for metric in all_metrics:
    for example in corpus:
        user_pref = example["user_preference"][metric] 
        if user_pref is None:
            user_pref = "Neutral"
        else:
            user_pref = example["generated_citations"][user_pref]["model"]
        
        for model_name in results[metric]:
            if model_name == user_pref:
                results[metric][model_name].append(1)
            else:
                results[metric][model_name].append(0)

In [8]:
print( "\t".join( [ model_A,  "Neutral", model_B ] ) )

gpt-3.5-turbo-0301	Neutral	../rl_training/ppo_model/galactica-6.7b-ct2


In [9]:
for metric in all_metrics:
    print(metric+":")
    print( " & ".join( [  "%.1f"%(np.round(np.mean( results[metric][model_name] ) * 100 ,1))  for model_name in [ model_A,  "Neutral", model_B ] ] )  )
    
    res = wilcoxon( np.array( results[metric][model_A] ) -  np.array( results[metric][model_B] ) )
    print(res)

IAS:
17.1 & 57.1 & 25.7
WilcoxonResult(statistic=414.0, pvalue=0.17971249487899976)
KR:
15.2 & 67.6 & 17.1
WilcoxonResult(statistic=280.0, pvalue=0.7316005889599013)
FS:
34.3 & 50.5 & 15.2
WilcoxonResult(statistic=424.0, pvalue=0.005545667315244056)
Similarity:
24.8 & 30.5 & 44.8
WilcoxonResult(statistic=962.0, pvalue=0.013976605613143215)
